In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import re

import dask.dataframe as dd
import dask.array as da
# import dask.bag as db
import dask
from dask.distributed import Client, SSHCluster
import dask.distributed

# import hvplot.dask

In [2]:
cluster = SSHCluster(
    [ "bhbh-1", "bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1"],
    connect_options={"client_keys": "/home/ubuntu/private/tbertola_key.pem"},
    worker_options={"nthreads": 1}, # because each bhbh-* has 4 cores
    scheduler_options={"port": 8786, "dashboard_address": ":8787"}
)

2023-06-07 15:12:45,659 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:45,658 - distributed.scheduler - INFO - State start
2023-06-07 15:12:45,664 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:45,663 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.140:8786
2023-06-07 15:12:46,735 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:46,733 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:34089'
2023-06-07 15:12:46,812 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:46,810 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:46511'
2023-06-07 15:12:46,861 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:46,860 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:38167'
2023-06-07 15:12:46,963 - distributed.deploy.ssh - INFO - 2023-06-07 15:12:46,961 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:37241'
2023-06-07 15:12:47,578 - distributed.deploy.ssh - INFO - 2023-06-07

In [3]:
client = Client(cluster)

/home/ubuntu/miniconda3/lib/python3.10/site-packages/distributed/client.py:1386: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+------------------+
| Package | Client | Scheduler | Workers          |
+---------+--------+-----------+------------------+
| tornado | 6.3.2  | 6.3.2     | {'6.3.2', '6.2'} |
+---------+--------+-----------+------------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
client

<Client: 'tcp://10.67.22.140:8786' processes=8 threads=8, memory=15.53 GiB>

In [5]:
#dask.config.get('scheduler')

In [6]:
import glob
dir_path = '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_*'
dir_list = glob.glob(dir_path)
print(len(dir_list))

60


In [7]:
output_column_to_remove = ['ID', 'MHE_0', 'MCO_0', 'Radius_0', 'RHE_0','RCO_0', 'Luminosity_0', 'Temperature_0', 'Lambda_0',
                               'Phase_0', 'PhaseBSE_0', 'Zams_0', 'MHE_1', 'MCO_1','Radius_1', 'RHE_1', 'RCO_1', 
                               'Luminosity_1', 'Temperature_1','Lambda_1', 'Phase_1', 'PhaseBSE_1', 'Zams_1']
    
evolved_column_to_remove = ['#ID', 'spin_0', 'Tstart_0', 'spin_1', 'Tstart_1', 'Tend', 'Dtout', 'Seed']
drop_list = ['RemnantType_0',  'RemnantType_1']

In [8]:
def FGpreprocessing(dir_path: str, n_thread: int, output_column_to_remove: list, evolved_column_to_remove: list, drop_list: list):
    
    output_str = f'{dir_path}/0/output_{n_thread}.csv'
    evolved_str = f'{dir_path}/0/evolved_{n_thread}.dat'
    logfile_str = f'{dir_path}/0/logfile_{n_thread}.dat'
    
    alpha = float(re.findall(r".+(?<=A)(.*)(?=L)", output_str)[0])
    ################################
    output = dd.read_csv(output_str).\
                rename(columns={'Mass_0':'Mass_0_out', 'Mass_1':'Mass_1_out'}).\
                drop(columns=output_column_to_remove)


    idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
    output=output[idxBHBH] #Distributed DataFrame containing only binaries BB

    ################################         

    evolved = dd.read_table(evolved_str, sep='\s+').\
                drop(columns=evolved_column_to_remove)
    evolved['alpha'] = alpha
    ################################

    logfile = dd.read_csv(logfile_str, header=None)

    df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='RLO').\
                reset_index()

    df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='CE').\
                reset_index()

    df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='BSN').\
                reset_index()

    ################################

    bhbh = evolved.merge(output, on=['name'], how='inner').\
                   merge(df_RLO, on=['name'], how='left').\
                   merge(df_CE,  on=['name'], how='left').\
                   merge(df_BSN, on=['name'], how='left').\
                   fillna(value=0).\
                   drop(columns=drop_list)

    ###############################

    bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime']

    bhbh['Mass_max_out'] = bhbh['Mass_1_out']
    bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                            where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']), other=bhbh['Mass_0_out'])

    bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
    bhbh['q'] = bhbh['q'].\
                where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']), other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

    bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))
    
    return bhbh

## Dask delayed

In [9]:
%%time
n_threads_DEMO = 30
bhbh_list=[]
for dir_name in dir_list[:3]:
    for i in range(n_threads_DEMO):
        _ = dask.delayed(FGpreprocessing)(dir_name, i,  output_column_to_remove, evolved_column_to_remove, drop_list)
        bhbh_list.append(_)

CPU times: user 24 ms, sys: 3.27 ms, total: 27.3 ms
Wall time: 27.1 ms


In [10]:
%%time
results = dask.compute(*bhbh_list) 

CPU times: user 916 ms, sys: 119 ms, total: 1.04 s
Wall time: 7.06 s


In [11]:
print(type(results))
results

<class 'tuple'>


(Dask DataFrame Structure:
                   name   Mass_0      Z_0    SN_0   Mass_1      Z_1    SN_1        a        e    alpha Mass_0_out Mass_1_out Semimajor Eccentricity   GWtime BWorldtime BEvent    RLO     CE    BSN   tdelay Mass_max_out        q Mass_chirp
 npartitions=12                                                                                                                                                                                                                              
                 object  float64  float64  object  float64  float64  object  float64  float64  float64    float64    float64   float64      float64  float64    float64  int64  int64  int64  int64  float64      float64  float64    float64
                    ...      ...      ...     ...      ...      ...     ...      ...      ...      ...        ...        ...       ...          ...      ...        ...    ...    ...    ...    ...      ...          ...      ...        ...
 ...                .

In [12]:
%%time
results = dask.compute(*results)

CPU times: user 9.51 s, sys: 689 ms, total: 10.2 s
Wall time: 16min 41s


## Test no dask.delayed (only scheduler)

In [9]:
%%time
n_threads_DEMO = 30
bhbh_list=[]
for dir_name in dir_list[:3]:
    for i in range(n_threads_DEMO):
        _ = FGpreprocessing(dir_name, i,  output_column_to_remove, evolved_column_to_remove, drop_list)
        bhbh_list.append(_)

CPU times: user 31.7 s, sys: 1.33 s, total: 33.1 s
Wall time: 54.2 s


In [10]:
type(bhbh_list)

list

In [11]:
bhbh_list

[Dask DataFrame Structure:
                   name   Mass_0      Z_0    SN_0   Mass_1      Z_1    SN_1        a        e    alpha Mass_0_out Mass_1_out Semimajor Eccentricity   GWtime BWorldtime BEvent    RLO     CE    BSN   tdelay Mass_max_out        q Mass_chirp
 npartitions=12                                                                                                                                                                                                                              
                 object  float64  float64  object  float64  float64  object  float64  float64  float64    float64    float64   float64      float64  float64    float64  int64  int64  int64  int64  float64      float64  float64    float64
                    ...      ...      ...     ...      ...      ...     ...      ...      ...      ...        ...        ...       ...          ...      ...        ...    ...    ...    ...    ...      ...          ...      ...        ...
 ...                .

In [12]:
%%time

results = dask.compute(*bhbh_list)

CPU times: user 10.7 s, sys: 660 ms, total: 11.4 s
Wall time: 19min 31s


## For loop in each directory 

In [13]:
# alpha = [float(re.findall(r".+(?<=A)(.*)(?=L)", dir_)[0]) for dir_ in dir_list]

def FG_new(alpha: float,
           dir_path: str,
#            n_thread: int,
           output_column_to_remove: list,
           evolved_column_to_remove: list,
           drop_list: list):
    
    output_str = f'{dir_path}/0/output_*.csv'
    evolved_str = f'{dir_path}/0/evolved_*.dat'
    logfile_str = f'{dir_path}/0/logfile_*.dat'

    ################################
    output = dd.read_csv(output_str).\
                rename(columns={'Mass_0':'Mass_0_out', 'Mass_1':'Mass_1_out'}).\
                drop(columns=output_column_to_remove)


    idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
    output=output[idxBHBH] #Distributed DataFrame containing only binaries BB

    ################################   

    
    dask_divisions = output.set_index("name").divisions
    unique_divisions = list(dict.fromkeys(list(dask_divisions)))
    
    
    ##############
    
    output=output.set_index("name", divisions=unique_divisions)

    ############

    evolved = dd.read_table(evolved_str, sep='\s+').\
                drop(columns=evolved_column_to_remove).set_index("name", divisions=unique_divisions)
    ################################

    logfile = dd.read_csv(logfile_str, header=None)

    df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='RLO').\
                reset_index().set_index('name', divisions=unique_divisions)

    df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='CE').\
                reset_index().set_index('name', divisions=unique_divisions)

    df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='BSN').\
                reset_index().set_index('name', divisions=unique_divisions)

    ################################

    bhbh = evolved.merge(output, left_index=True, right_index=True, how='inner').\
                   merge(df_RLO, left_index=True, right_index=True, how='left').\
                   merge(df_CE,  left_index=True, right_index=True, how='left').\
                   merge(df_BSN, left_index=True, right_index=True, how='left').\
                   fillna(value=0).\
                   drop(columns=drop_list)

    ###############################

    bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime']

    bhbh['Mass_max_out'] = bhbh['Mass_1_out']
    bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                            where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']), other=bhbh['Mass_0_out'])

    bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
    bhbh['q'] = bhbh['q'].\
                where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']), other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

    bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))
    bhbh['alpha']=alpha
#     bhbh=bhbh.compute()
    return bhbh

In [14]:
alpha = [float(re.findall(r".+(?<=A)(.*)(?=L)", dir_)[0]) for dir_ in dir_list[:3]]

### Distributed because of reset index (?)

In [15]:
%%time 

bhbh_list=[]
for i,directory in enumerate(dir_list[:3]):
    print(i)
    bhbh_list.append(FG_new(alpha[i], directory, output_column_to_remove, evolved_column_to_remove, drop_list))

0
1
2
CPU times: user 5.65 s, sys: 566 ms, total: 6.22 s
Wall time: 12min 28s


In [16]:
type(bhbh_list)

list

In [17]:
bhbh_list

[Dask DataFrame Structure:
                     Mass_0      Z_0    SN_0   Mass_1      Z_1    SN_1        a        e Mass_0_out Mass_1_out Semimajor Eccentricity   GWtime BWorldtime BEvent    RLO     CE    BSN   tdelay Mass_max_out        q Mass_chirp    alpha
 npartitions=360                                                                                                                                                                                                                        
 0_100000507552376  float64  float64  object  float64  float64  object  float64  float64    float64    float64   float64      float64  float64    float64  int64  int64  int64  int64  float64      float64  float64    float64  float64
 0_101123165891357      ...      ...     ...      ...      ...     ...      ...      ...        ...        ...       ...          ...      ...        ...    ...    ...    ...    ...      ...          ...      ...        ...      ...
 ...                    ...      ...     

In [18]:
%%time 

for i in range(len(bhbh_list[:3])):
    bhbh_list[i].to_parquet('/mnt/bhbh/a_new_hope/test'+str(i)+'.parquet')

CPU times: user 2.95 s, sys: 229 ms, total: 3.18 s
Wall time: 17min 18s


## Dask delayed (all directory)

In [9]:
%%time
n_threads_DEMO = 30
bhbh_list=[]
for dir_name in dir_list:
    for i in range(n_threads_DEMO):
        _ = dask.delayed(FGpreprocessing)(dir_name, i,  output_column_to_remove, evolved_column_to_remove, drop_list)
        bhbh_list.append(_)

CPU times: user 501 ms, sys: 37.1 ms, total: 538 ms
Wall time: 550 ms


In [10]:
%%time
results = dask.compute(*bhbh_list) 

CPU times: user 17.7 s, sys: 2.62 s, total: 20.3 s
Wall time: 2min 12s


In [11]:
print(type(results))
results[:2]

<class 'tuple'>


(Dask DataFrame Structure:
                   name   Mass_0      Z_0    SN_0   Mass_1      Z_1    SN_1        a        e    alpha Mass_0_out Mass_1_out Semimajor Eccentricity   GWtime BWorldtime BEvent    RLO     CE    BSN   tdelay Mass_max_out        q Mass_chirp
 npartitions=12                                                                                                                                                                                                                              
                 object  float64  float64  object  float64  float64  object  float64  float64  float64    float64    float64   float64      float64  float64    float64  int64  int64  int64  int64  float64      float64  float64    float64
                    ...      ...      ...     ...      ...      ...     ...      ...      ...      ...        ...        ...       ...          ...      ...        ...    ...    ...    ...    ...      ...          ...      ...        ...
 ...                .

In [ ]:
%%time
results = dask.compute(*results)